# <a id='Урок_14'>Домашнее задание № 13. Модель BERT и GPT</a>

1. Взять датасет https://huggingface.co/datasets/merionum/ru_paraphraser решить задачу парафраза

2. (дополнительно необязательная задача)на выбор взять https://huggingface.co/datasets/sberquad https://huggingface.co/datasets/blinoff/medical_qa_ru_data натренировать любую модель для вопросно ответной системы как альтернатива можно взять любой NER датасет из https://github.com/natasha/corus#reference и обучить NER

## <a id='Подготовка_14'>Подготовка</a>

In [1]:
!pip install datasets
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 KB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 KB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 KB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.1/199.1 KB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 KB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 20.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 KB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 KB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 KB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 11.6 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/si

In [2]:
#!g2.2
# Импорт библиотек и модулей
import pandas as pd
import numpy as np
from datasets import load_dataset, Dataset, DatasetDict
from datasets import load_metric
from sklearn import preprocessing
from transformers import AutoModelForSequenceClassification, BertTokenizer
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer
from transformers import DataCollatorForTokenClassification
import torch
import logging
from transformers.trainer import logger as noisy_logger

## <a id='Загрузка_данных_13'>Загрузка и предобработка данных</a>

In [ ]:
#!g2.2
corpus_train = load_dataset('merionum/ru_paraphraser', data_files='train.jsonl')
corpus_test = load_dataset('merionum/ru_paraphraser', data_files='test.jsonl')

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset json downloaded and prepared to /tmp/xdg_cache/huggingface/datasets/merionum___json/merionum--ru_paraphraser-dd0dcf23f5ec66a4/0.0.0/0f7e3662623656454fcd2b650f34e886a7db4b9104504885bd462096cc7a9f51. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset json downloaded and prepared to /tmp/xdg_cache/huggingface/datasets/merionum___json/merionum--ru_paraphraser-62b622e3b2274f4e/0.0.0/0f7e3662623656454fcd2b650f34e886a7db4b9104504885bd462096cc7a9f51. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
#!g2.2
df_train = pd.DataFrame(corpus_train['train'])
df_train['text'] =  list(zip(df_train['text_1'], df_train['text_2']))
df_train = df_train[:2000]
df_train.head(3)

,id,id_1,id_2,text_1,text_2,class,text
0,1,201,8159,Полицейским разрешат стрелять на поражение по ...,Полиции могут разрешить стрелять по хулиганам ...,0,(Полицейским разрешат стрелять на поражение по...
1,2,202,8158,Право полицейских на проникновение в жилище ре...,Правила внесудебного проникновения полицейских...,0,(Право полицейских на проникновение в жилище р...
2,3,273,8167,Президент Египта ввел чрезвычайное положение в...,Власти Египта угрожают ввести в стране чрезвыч...,0,(Президент Египта ввел чрезвычайное положение ...


In [ ]:
#!g2.2
df_test = pd.DataFrame(corpus_test['train'])
df_test['text'] =  list(zip(df_test['text_1'], df_test['text_2']))
df_test = df_test[:500]
df_test.head(3)

,id,id_1,id_2,text_1,text_2,class,text
0,25349,34420,34421,Цены на нефть восстанавливаются,Парламент Словакии поблагодарил народы бывшего...,-1,"(Цены на нефть восстанавливаются, Парламент Сл..."
1,25360,34438,34439,"""Гоголь-центр"" покажет видеозапись скандальног...",Кехман запретил «Гоголь-центру» показывать вид...,-1,"(""Гоголь-центр"" покажет видеозапись скандально..."
2,25382,34475,34476,Агент: РФС вновь задерживает зарплату Фабио Ка...,СМИ: Агент Фабио Капелло грозится подать в суд...,-1,(Агент: РФС вновь задерживает зарплату Фабио К...


In [ ]:
#!g2.2
model_name = 'cointegrated/rubert-base-cased-dp-paraphrase-detection'
tokenizer = BertTokenizer.from_pretrained(model_name)

Downloading:   0%|          | 0.00/1.57M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/406 [00:00<?, ?B/s]

In [ ]:
#!g2.2
text1 = df_test['text_1'][0]
text2 = df_test['text_2'][0]
tokenizer(text1, text2)

{'input_ids': [101, 59713, 1469, 20264, 26631, 5052, 102, 28362, 27443, 57650, 37777, 12678, 5576, 1758, 50938, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [ ]:
#!g2.2
le = preprocessing.LabelEncoder()
le.fit(df_train['class'])

df_train['class'] = le.transform(df_train['class']) #array([1, 1, 1, ..., 0, 0, 0])
df_test['class'] = le.transform(df_test['class'])

In [ ]:
#!g2.2
df_train.rename(columns = {'class': 'label'}, inplace = True)
df_test.rename(columns = {'class': 'label'}, inplace = True)

In [ ]:
#!g2.2
data = DatasetDict({
    'train': Dataset.from_pandas(pd.DataFrame(df_train[['text_1', 'text_2', 'label']])),
    'test': Dataset.from_pandas(pd.DataFrame(df_test[['text_1', 'text_2', 'label']]))
})

In [ ]:
#!g2.2
data['train']

Dataset({
    features: ['text_1', 'text_2', 'label'],
    num_rows: 2000
})

In [ ]:
#!g2.2
def tokenize_function(examples):
    return tokenizer(examples["text_1"], examples["text_2"], padding="max_length", truncation=True, max_length = 22)
  # return tokenizer(examples["text"], padding="max_length", truncation=True, max_length = 22)

In [ ]:
#!g2.2
tokenized_datasets = data.map(tokenize_function, batched=True)

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pai

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pai

In [ ]:
#!g2.2
num_labels = df_test['label'].nunique()
num_labels

3

In [ ]:
#!g2.2
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels = num_labels, ignore_mismatched_sizes=True)

Downloading:   0%|          | 0.00/1.09k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/679M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at cointegrated/rubert-base-cased-dp-paraphrase-detection and are newly initialized because the shapes did not match:
- classifier.weight: found shape torch.Size([2, 768]) in the checkpoint and torch.Size([3, 768]) in the model instantiated
- classifier.bias: found shape torch.Size([2]) in the checkpoint and torch.Size([3]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
#!g2.2
batch_size = 128

# Определяем аргументы
args = TrainingArguments(
    'classification',
    evaluation_strategy = "epoch",
    learning_rate=1e-6,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=50,
    # weight_decay=0.01,
    save_strategy='no',
    report_to='none',
)

# args = TrainingArguments(output_dir="test_trainer")

In [ ]:
#!g2.2
data_collator = DataCollatorForTokenClassification(tokenizer)

In [ ]:
#!g2.2
metric = load_metric("accuracy")

<ipython-input-18-e56166b75a9d>:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("accuracy")


In [ ]:
#!g2.2
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [ ]:
#!g2.2
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['test'],
    compute_metrics = compute_metrics,
    # data_collator=data_collator, # в классификации не требуется
    # tokenizer=tokenizer,
)

In [ ]:
#!g2.2
for param in model.parameters():
    param.requires_grad = True

In [ ]:
#!g2.2
noisy_logger.setLevel(logging.WARNING)

In [ ]:
#!g2.2
trainer.train()

/usr/local/lib/python3.8/dist-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.924771,0.622000
2,No log,0.918491,0.618000
3,No log,0.911653,0.612000
4,No log,0.904643,0.610000
5,No log,0.896038,0.628000
6,No log,0.887889,0.642000
7,No log,0.882036,0.652000
8,No log,0.876418,0.642000
9,No log,0.871527,0.634000
10,No log,0.866649,0.616000


/usr/local/lib/python3.8/dist-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/usr/local/lib/python3.8/dist-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/usr/local/lib/python3.8/dist-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/usr/local/lib/python3.8/dist-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqu

TrainOutput(global_step=400, training_loss=0.77412841796875, metrics={'train_runtime': 53.5733, 'train_samples_per_second': 1866.602, 'train_steps_per_second': 7.466, 'total_flos': 1130565466800000.0, 'train_loss': 0.77412841796875, 'epoch': 50.0})

In [ ]:
#!g2.2
trainer.evaluate()

/usr/local/lib/python3.8/dist-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


/usr/local/lib/python3.8/dist-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


{'eval_loss': 0.8335411548614502,
 'eval_accuracy': 0.632,
 'eval_runtime': 0.1161,
 'eval_samples_per_second': 4305.454,
 'eval_steps_per_second': 17.222,
 'epoch': 50.0}

In [ ]:
#!g2.2
predictions, labels, _ = trainer.predict(tokenized_datasets["test"])
predictions = np.argmax(predictions, axis=-1)
metric.compute(predictions=predictions, references=labels)

/usr/local/lib/python3.8/dist-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/usr/local/lib/python3.8/dist-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


{'accuracy': 0.632}

In [ ]:
#!g2.2
text_1 = df_test['text_1'][3]
text_2 = df_test['text_2'][3]
text_1, text_2

('День Победы в Москве обещает выдаться облачным',
 'Любляна отпразднует День Победы вместе с Москвой')

In [ ]:
#!g2.2
tokens = tokenizer(text_1, text_2,  padding="max_length", truncation=True, max_length = 22, return_tensors='pt')

In [ ]:
#!g2.2
tokens = {k: v for k, v in tokens.items()}

In [ ]:
#!g2.2
with torch.no_grad():
    pred = model(**tokens)

RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu! (when checking arugment for argument index in method wrapper_index_select)

In [ ]:
#!g2.2
print(f'Истина: {df_test["label"][3]}, \nПредсказание:{pred.logits.argmax(dim=-1)[0].cpu().numpy()}')